<span style="font-family:Lucida Bright;">

<hr style="border:2px solid black"> </hr>

<p style="margin-bottom:1cm"></p>

<center>
<font size="7"><b>Social Data Analysis and Visualization</b></font>
<p style="margin-bottom:1cm"></p>
<font size="6.8"><b>Final Project</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="5.8"><b>Data Import and Cleaning</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<br>
<font size="3"><b></b></font>

</center>

<p style="margin-bottom:0.7cm"></p>

<hr style="border:2px solid black"> </hr>

<hr style="border:2px solid black"> </hr>

# Initialization

## How to read this notebook

<span style="font-family:Arial;">

Please note that the pre-rendered outputs will first display properly when the notebook is __trusted__.

## Imports

In [1]:
%matplotlib inline

import bokeh.plotting as bplt
import calendar
import datetime
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import scipy.stats
import seaborn as sns
import urllib.request
import warnings

from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.models import Legend
from bokeh.models.ranges import FactorRange
from bokeh.models.sources import ColumnDataSource
from folium.map import FeatureGroup
from folium.plugins import HeatMap, HeatMapWithTime
from functools import reduce
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
from IPython.display import Markdown
from IPython.display import YouTubeVideo
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.image import NonUniformImage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from operator import itemgetter
from pathlib import Path
from scipy import stats
from sklearn.datasets import fetch_20newsgroups
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

##  Configuration

### Notebook options

In [ ]:
# Decide which output is shown below the cells.
InteractiveShell.ast_node_interactivity = "none"

### Bokeh options

In [ ]:
# Show bokeh figures in the notebook.
output_notebook()

### Matplotlib options

In [50]:
# Show matplotlib plots inline.
%matplotlib inline

### Pandas options

In [2]:
# Define the format in which the numbers will be shown in
# the pandas dataframes.
pd.options.display.float_format = '{:,.1f}'.format

# Decide how to handle the "SettingWithCopyWarning" warning
pd.options.mode.chained_assignment = None  # default='warn'

# Set the maximum number of rows and columns to show when 
# displaying a Pandas dataframe.
pd.options.display.max_rows = 75
pd.options.display.max_columns = 75

Loading BokehJS ...

### Warnings

In [ ]:
# Decide how to handle warnings.
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=PerformanceWarning)

## Paths

In [3]:
# Project root.
path_root = Path(
    r'C:\GDrive\DTU\Kurser\Social_Data_Analysis_and_Visualization_02806\final_project')

# Resources root.
path_resources_root = path_root / 'resources'

# Data folders.
path_data_root = path_resources_root / 'data'
path_data_citizenship_root = path_data_root / 'citizenship'
path_data_dwellings_root = path_data_root / 'dwellings'
path_data_geo_root = path_data_root / 'geo'
path_data_marital_status_root = path_data_root / 'marital_status'
path_data_without_district_info_root = path_data_root / 'without_district_information'
path_data_clean_root = path_data_root / 'clean'

## Function definitions

### General functions

In [4]:
# A function that returns a dict of object attributes.
def get_obj_attributes(obj):
    return {attribute_name: getattr(obj, attribute_name)
            for attribute_name in dir(obj)
            if (not attribute_name.startswith('__')
                and not callable(getattr(obj, attribute_name)))}

# A function that will print a markdown text.
def printmd(string):
    display(Markdown(string))
    
# A function that returns unique values from a text.
def unique(list_):
    return list(set(list_))

### Load data

In [51]:
# Define a class th
class CphData:

    def __init__(self):
        # Country of origin (no distric info)
        df_country_of_origin = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_country_of_origin_without_district.pkl')
        df_country_of_origin.attrs = {'name': 'Country of origin'}
        self.country_of_origin = df_country_of_origin

        # Citizenship
        df_citizenship = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_citizenship.pkl')
        df_citizenship.attrs = {'name': 'Citizenship'}
        self.citizenship = df_citizenship

        # Marital status
        df_marital_status = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_marital_status.pkl')
        df_marital_status.attrs = {'name': 'Marital status'}
        self.marital_status = df_marital_status

        # Family type and children
        df_family_type_and_children = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_family_type_and_number_of_chidren.pkl')
        df_family_type_and_children.attrs = {
            'name': 'Family type and children'}
        self.family_type_and_children = df_family_type_and_children

        # Income
        df_income = pd.read_pickle(path_data_clean_root / 'cph_income.pkl')
        df_income.attrs = {'name': 'Income'}
        self.income = df_income

        # Life span
        df_life_span = pd.read_pickle(
            path_data_clean_root / 'cph_life_span.pkl')
        df_life_span.attrs = {'name': 'Life span'}
        self.life_span = df_life_span

        # Population movement
        df_population_movement = pd.read_pickle(
            path_data_clean_root / 'cph_population_movement.pkl')
        df_population_movement.attrs = {'name': 'Population movement'}
        self.population_movement = df_population_movement

        # Dwellings
        df_dwellings = pd.read_pickle(
            path_data_clean_root / 'cph_dwellings.pkl')
        df_dwellings.attrs = {'name': 'Dwellings'}
        self.dwellings = df_dwellings

    # Gets dataframes with district information:
    def get_dataframes_with_district(self):
        return [df
                for df in self.get_all_dataframes()
                if 'District' in df.columns]

    # Gets all the dataframes and returns them in a list.
    def get_all_dataframes(self):
        return list(get_obj_attributes(self).values())

    # Gets the names of all columns in all datasets.
    def get_all_column_names(self):
        # Initialize the list of all columns from all the dataframes:
        all_columns = list()

        # Get all columns.
        for df in self.get_all_dataframes():
            all_columns += df.columns.to_list()

        return list(sorted(set(all_columns)))

    # Displays the dataframes with specified names.
    def display_dataframes(self, names=None):
        for attribute_name, df in get_obj_attributes(self).items():
            if (names is None) or (df.attrs["name"] in names):
                printmd(f'\n**{df.attrs["name"]}**:')
                display(df)

    # Gets the names of the dataframes in the object.
    def get_dataframe_names(self):
        return [df.attrs['name'] for df in self.get_dataframes()]

    # Displays the names of the dataframes in the object.
    def display_dataframe_names(self):
        # Display all the dataframes.
        df_names = self.get_dataframe_names()
        printmd(f'The dataframes in the object are:')
        printmd(f'***{"***, ***".join(df_names)}***')

### Plotting functions

In [6]:
# A function that applies default formatting to an axes.
def format_axes(axes: plt.Axes,
                keep_box=False):
    if not keep_box:
        axes.spines['top'].set_color('white')
        axes.spines['right'].set_color('white')

    axes.set_facecolor("white")


# A function that applies default formatting to annotation
# of an axes.
def format_axes_annotation(axes: plt.Axes):
    axes.xaxis.label.set_fontsize(14)
    axes.yaxis.label.set_fontsize(14)
    axes.title.set_fontsize(16)


# A function for creating common x-label for the figure.
def figure_x_label(figure: plt.Figure,
                   label: str,
                   y_position=0.04,
                   font_size=16):
    figure.text(0.5, y_position, label,
                ha='center',
                fontdict={'size': font_size})


# A function for creating common y-label for the figure.
def figure_y_label(figure: plt.Figure,
                   label: str,
                   x_position=0.04,
                   font_size=16):
    figure.text(x_position, 0.5, label,
                va='center',
                rotation='vertical',
                fontdict={'size': font_size})


# A function that draws a horizontal line across the entire axes.
def draw_threshold(value: float,
                   axes: plt.Axes,
                   linewidth=1,
                   linestyle='-',
                   color=None,
                   title=None):
    
    # Get axes limits and ranges.
    x_min, x_max = axes.get_xlim()
    x_range = x_max - x_min
    y_min, y_max = axes.get_ylim()
    y_range = y_max - y_min
    
    # Plot the threshold line.
    axes.plot([x_min, x_max], [value, value],
              linewidth=1,
              linestyle='-',
              color=color)
    
    # Write a title above the threshold line
    if title is not None:
        axes.text(x_min + 0.01 * x_range,
                  value + 0.02 * y_range,
                  title)

###  Dataframe functions

In [7]:
# A function that gets column names of a dataframe.
def get_df_columns(df, exclude=None):
    # If columns to be excluded have not been defined,
    # represent it as an empty list.
    if exclude is None:
        exclude = list()

    # If the columns to be excluded are not specified using a list
    # or a tuple, represent them as a list.
    elif not isinstance(exclude, (list, tuple)):
        exclude = [exclude]

    # Return all column names except the ones to exclude.
    return [column for column in df.columns.to_list()
            if column not in exclude]

# A function that resets the names of indices
def reset_df_index_names(df):
    return (
        df
        .rename_axis(['' for level in range(df.columns.nlevels)],
                     axis="columns")
        .rename_axis(['' for level in range(df.index.nlevels)],
                     axis="rows")
    )


# A function for balancing a dataframe so that the number of rows
# containing each value present in the designated column will be the same.
def balance_dataframe(df: pd.DataFrame, column_name):
    # Get the number of crimes for the least frequent crime.
    lowest_frequency = df['Category'].value_counts().min()

    # Create an empty dataframe for storing the balanced data
    df_balanced = pd.DataFrame()

    # For each value in column, randomly choose the number of samples
    # that corresponds to the least frequent value in the column.
    for value in df[column_name].unique():
        df_balanced = df_balanced.append(
            df
            .loc[df[column_name] == value]
            .sample(lowest_frequency)
        )

    return df_balanced


# A function that evaluates a dictionary of models on data from
# a pandas dataframe.
def evaluate_models(models: dict,
                    df: pd.DataFrame,
                    predictor_labels: list,
                    target_label: str,
                    test_size=0.33):

    # Get the dataset.
    X = df.loc[:, predictor_labels].values
    y = df.loc[:, target_label].values

    # Split the dataset into a test and training set.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=32)

    # Fit the models to the data.
    for model_name, model in models.items():

        # Print the name of the model.
        printmd(f'*__{model_name}:__*')

        # Train the model on the training set.
        model.fit(X_train, y_train)

        # Get the predictions on the test set.
        predictions = model.predict(X_test)

        # Print the classification report.
        print(classification_report(y_test, predictions,
                                    zero_division=0,
                                    digits=4))


# A function that sorts the columns in alphabethical order
# and puts the user-chosen columns first
def df_sort_columns(df: pd.DataFrame,
                    first_columns=['Year',
                                   'Quarter',
                                   'District',
                                   'District type',
                                   'Sex',
                                   'Age']):

    # Define a function that moves the chosen element to the
    # front of the list.
    def move_to_front(element, list_):
        if element in list_:
            list_.insert(0, list_.pop(list_.index(element)))

    # Make sure that the columns that are to be put in front
    # are represented as a list.
    if first_columns is None:
        first_columns = list()

    # Sort the columns in alphabetical order.
    sorted_columns = list(df.columns)
    sorted_columns.sort()

    # Move the user-chosen columns to the front.
    for column in first_columns[::-1]:
        move_to_front(column, sorted_columns)

    # Assign the ordered columns to the dataframe.
    df = df[sorted_columns]

    return df


# A function that creates a new column representing data in 'value_columns'
# for every unique value in 'category_columns'.
def df_create_column_for_each_unique_value(df,
                                           category_columns,
                                           value_columns,
                                           aggfunc='first'):

    # Always represent category and value columns as a list or tuple.
    if not isinstance(category_columns, (list, tuple)):
        category_columns = [category_columns]

    if not isinstance(value_columns, (list, tuple)):
        value_columns = [value_columns]

    # Create a colum order for grouping so that all the value columns
    # come last and category columns second last. We leave out 1
    # value column for the result
    cat_and_value_columns = category_columns + value_columns
    column_order = (
        get_df_columns(df, exclude=cat_and_value_columns)
        + cat_and_value_columns[:-1]
    )

    # Create columns from unique values by grouping and unstacking.
    df = (
        df
        .groupby(column_order)
        .first()
        .unstack(list(np.arange(-len(cat_and_value_columns) + 1,
                                0)))
        .reset_index()
    )

    # Delete the names of the index levels
    df = df.rename_axis(['' for level in range(df.columns.nlevels)],
                        axis="columns")
    return df

# OLD IMPLEMENTATION
#     return (
#         df
#         .pivot_table(values=value_columns,
#                      index=get_df_columns(df,
#                                           exclude=category_columns + value_columns),
#                      columns=category_columns,
#                      aggfunc='first')
#         .reset_index()
#         .rename_axis(('', ''), axis="columns")
#     )

###  Data cleaning functions

In [8]:
# Define a function that loads a dataframe from
# kk.statistikbank.dk
def load_cph_df(path_csv):
    return pd.read_csv(path_csv,
                       sep='\t',
                       skiprows=0,
                       encoding='windows-1252')


# A function that loads a dataframe from multiple files.
def load_split_dataframe(paths_csv):

    # Load the first two dataframes.
    df_1 = load_cph_df(paths_csv[0])
    df_2 = load_cph_df(paths_csv[1])

    # Extract common columns.
    common_columns = list(set(df_1.columns)
                          & set(df_2.columns))

    # Load the whole dataframe.
    for idx, path_csv in enumerate(paths_csv):
        # Merge the dataframes.
        if idx == 0:
            df = load_cph_df(path_csv)
        else:
            df = df.merge(load_cph_df(path_csv),
                          left_on=common_columns,
                          right_on=common_columns)

    # Return the merged dataframe.
    return df


# A function that cleans the data about copenhagen district
def clean_districts(df: pd.DataFrame):

    # Continue only if the dataframe contains the column: "district"
    if 'district' not in df.columns:
        return df

    df = df.copy()

    # Define the dame of the column containing info about districts.
    district_column_name = 'district'

    # Extract the district names.
    districts = df[district_column_name]

    # Initialize the list for storing district types.
    district_types = list()

    # Define valid districts.
    valid_districts = ['Indre By', 'Østerbro', 'Nørrebro', 'Vesterbro/Kongens Enghave', 'Valby',
                       'Vanløse', 'Brønshøj-Husum', 'Bispebjerg', 'Amager Øst', 'Amager Vest']

    # Define valid districts.
    valid_polling_areas = ['1. Østerbro', '1. Øst', '1. Nord', '1. Syd', '1. Vest', '1. Nordvest',
                           '2. Sundbyvester', '2. Nord', '2. Syd', '2. Øst', '2. Vest', '3. Indre By',
                           '3. Nord', '3. Syd', '3. Øst',
                           '4. Sundbyøster', '4. Nord', '4. Syd', '4. Øst',
                           '5. Nørrebro', '5. Nord', '5. Nørrebrohallen',
                           '5. Syd', '5. Øst', '5. Vest', '5. Nordvest',
                           '6. Bispebjerg', '6. Vest', '6. Nord', '6. Øst', '6. Syd',
                           '7. Brønshøj', '7. Nord', '7. Syd', '7. Øst', '7. Vest',
                           '7. Nordvest', '7. Katrinedal', '7. Kirkebjerg', '7. Vanløse',
                           '8. Valby', '8. Nord', '8. Syd', '8. Vest', '8. Sydøst', '8. Midt',
                           '9. Vesterbro', '9. Nord', '9. Syd',
                           '9. Øst', '9. Vest', '9. Midt', '9. Sydhavn']

    # Assign type to each district.
    for district in districts:
        if (('District' in district) | (district in valid_districts)):
            district_types.append('District')

        elif (('Polling area' in district) | (district in valid_polling_areas)):
            district_types.append('Polling area')

        elif 'Copenhagen total' in district:
            district_types.append('Entire Copenhagen')

        else:
            district_types.append('Unknown')

    # Insert the column "district type" next to the district column.
    idx_district_column = df.columns.to_list().index(district_column_name)
    df.insert(idx_district_column + 1, 'district type', district_types)

    # Clean the names in the district - column.
    for token in ['District -', 'Polling area -']:
        df[district_column_name] = (
            df[district_column_name]
            .str.replace(token, '')
            .str.strip()
        )

    return df


# A function that cleans all the information about the time in the
# dataframes for Copenhagen.
def clean_years(df: pd.DataFrame,
                non_year_columns=None,
                value_name='Value'):

    if non_year_columns is not None:
        # If "non_year_columns" is an integer, regard the first n
        # columns represented by the integer as non-year columns
        if isinstance(non_year_columns, int):
            non_year_columns = df.columns[:non_year_columns]
    else:
        non_year_columns = [column for column in df.columns.to_list()
                            if column[0] not in ['1', '2']]

    # Create a row for each Year and Quarter.
    df = df.melt(
        id_vars=non_year_columns,
        var_name="Time",
        value_name=value_name
    )

    # If the time is represented by a year and a quarter,
    # create columns "Year" and "Quarter" from the column "Time".
    if 'Q' in df.loc[0, 'Time']:
        df[['Year', 'Quarter']] = (
            df
            .pop('Time').str.split('Q', 2, expand=True)
            .astype(int)
        )

    # If year is presented as "XXXX:YYYY", take the second value.
    elif ':' in df.loc[0, 'Time']:
        df['Year'] = (
            df
            .pop('Time')
            .str.split(':').str[-1]
            .astype(int)
        )

    # Otherwise, transform the year into an integer
    else:
        df['Year'] = df.pop('Time').astype(int)

    return df


# A fuction that removes all other quarters of the year than last.
def choose_latest_quarter(df: pd.DataFrame, value_name):
    # Run only if the dataframe has a column "Age"

    if 'Quarter' not in df.columns:
        return df

    return (
        df
        .sort_values(by=['Year', 'Quarter'])
        .groupby([column for column in df.columns
                  if column not in ['Quarter', value_name]],
                 as_index=False)
        .last()
    )


# A function that perfoms standard cleaning of a dataframe
# from kk.statistikbank.dk
def clean_cph_dataframe(df, value_name='Value', df_name=''):
    # Clean district information.
    df = clean_districts(df)

    # Clean time information.
    df = clean_years(df, value_name=value_name)

    # Capitalize column names.
    df.columns = [column.capitalize() for column in df.columns]

    # Standardize age intervals
    df_standardize_age(df, value_name)

    # Choose only the last available quarter of the year.
    df = choose_latest_quarter(df, value_name)  
    
    # Order the columns.
    df = df_sort_columns(df)
    
    # Name the dataframe.
    df.name = df_name

    return df


# A function that displays all basic stats about the dataframe.
def show_stats(df):
    # Show data types.
    display(df.dtypes.to_frame('Data types'))

    # Show missing values.
    display(
        df.isna()
        .sum()
        .to_frame('Number of missing values')
    )

    # Describe values in columns.
    display(df.describe(include='all'))


# A function that unifies age intervals in the databases from
# from Københavns Kommune:
def df_standardize_age(df: pd.DataFrame, sum_column):

    # Run only if the dataframe has a column "Age"
    if 'Age' not in df.columns:
        return df

    # Use 10-year intervals for age: Create the mapping.
    mapping_5_to_10_years = dict()
    for interval_min in range(0, 90, 10):
        map_from_low = f'{interval_min}-{interval_min + 4} years'
        map_from_high = f'{interval_min + 5}-{interval_min + 9} years'
        map_to = f'{interval_min}-{interval_min + 9}'

        mapping_5_to_10_years[map_from_low] = map_to
        mapping_5_to_10_years[map_from_high] = map_to

    # Define the mapping for all the possible versions of the 90+ age.
    mapping_5_to_10_years['90-99 + years'] = '90+'
    mapping_5_to_10_years['95-99 years'] = '90+'
    mapping_5_to_10_years['90-94 years'] = '90+'
    mapping_5_to_10_years['100 years and over'] = '90+'
    mapping_5_to_10_years['95+years'] = '90+'

    # Apply the mapping to the column "Age".
    df['Age'] = df['Age'].map(mapping_5_to_10_years)

    # Sum the values in the new bins. First, get the names
    # of all the columns that are NOT the column which will summed up.
    non_sum_columns = [column for column in df.columns
                       if column != sum_column]

    # Sum the values together in the age bins.
    df[sum_column] = (
        df
        .groupby(non_sum_columns)
        .transform('sum')
    )

    # Drop duplicate rows
    df.drop_duplicates(ignore_index=True, inplace=True)

    return df

<hr style="border:2px solid black"> </hr>

# Load data

In [9]:
# Load the clean data.
cph_data = CphData()   

# Display all dataframes:
cph_data.display_dataframes()



**Citizenship**:

Year  Quarter  District District type    Sex  Age        Citizenship  \
0       1980        1   1. Nord  Polling area    Men  0-9            Denmark   
1       1981        1   1. Nord  Polling area    Men  0-9            Denmark   
2       1982        1   1. Nord  Polling area    Men  0-9            Denmark   
3       1983        1   1. Nord  Polling area    Men  0-9            Denmark   
4       1984        1   1. Nord  Polling area    Men  0-9            Denmark   
...      ...      ...       ...           ...    ...  ...                ...   
157435  2016        4  Østerbro      District  Women  90+  Western countries   
157436  2017        4  Østerbro      District  Women  90+  Western countries   
157437  2018        4  Østerbro      District  Women  90+  Western countries   
157438  2019        4  Østerbro      District  Women  90+  Western countries   
157439  2020        4  Østerbro      District  Women  90+  Western countries   

        Number of people  
0                  265.0  
1                  278.0  
2                  265.0  
3                  249.0  
4                  249.0  
...                  ...  
157435               3.0  
157436               4.0  
157437               5.0  
157438               6.0  
157439               5.0  

[157440 rows x 8 columns]


**Country of origin**:

Year  Quarter    Sex  Age Country of origin  Number of people
0      2008        4    Men  0-9         Abu Dhabi               0.0
1      2009        4    Men  0-9         Abu Dhabi               0.0
2      2010        4    Men  0-9         Abu Dhabi               0.0
3      2011        4    Men  0-9         Abu Dhabi               0.0
4      2012        4    Men  0-9         Abu Dhabi               0.0
...     ...      ...    ...  ...               ...               ...
61875  2016        4  Women  90+          Zimbabwe               0.0
61876  2017        4  Women  90+          Zimbabwe               0.0
61877  2018        4  Women  90+          Zimbabwe               0.0
61878  2019        4  Women  90+          Zimbabwe               0.0
61879  2020        4  Women  90+          Zimbabwe               0.0

[61880 rows x 6 columns]


**Dwellings**:

...

Year                   District District type  Average life span
0    2009                   Indre By      District               79.4
1    2009                   Østerbro      District               78.4
2    2009                   Nørrebro      District               72.3
3    2009  Vesterbro/Kongens Enghave      District               73.5
4    2009                      Valby      District               75.9
..    ...                        ...           ...                ...
127  2020             Brønshøj-Husum      District               78.5
128  2020                 Bispebjerg      District               77.8
129  2020                 Amager Øst      District               81.0
130  2020                Amager Vest      District               78.7
131  2020                  Unlocated      District                0.0

[132 rows x 4 columns]


**Marital status**:

Year  Quarter  District District type    Sex  Age Marital status  \
0       1974        1   1. Nord  Polling area    Men  0-9       Divorced   
1       1975        1   1. Nord  Polling area    Men  0-9       Divorced   
2       1976        1   1. Nord  Polling area    Men  0-9       Divorced   
3       1977        1   1. Nord  Polling area    Men  0-9       Divorced   
4       1978        1   1. Nord  Polling area    Men  0-9       Divorced   
...      ...      ...       ...           ...    ...  ...            ...   
240635  2016        4  Østerbro      District  Women  90+        Widowed   
240636  2017        4  Østerbro      District  Women  90+        Widowed   
240637  2018        4  Østerbro      District  Women  90+        Widowed   
240638  2019        4  Østerbro      District  Women  90+        Widowed   
240639  2020        4  Østerbro      District  Women  90+        Widowed   

        Number of people  
0                    0.0  
1                    0.0  
2                    0.0  
3                    0.0  
4                    0.0  
...                  ...  
240635             235.0  
240636             220.0  
240637             211.0  
240638             208.0  
240639             199.0  

[240640 rows x 8 columns]


**Population movement**:

Year  District District type  Deaths  Emigrated  Immigrated  \
0     1975   1. Nord  Polling area   134.0      122.0        84.0   
1     1976   1. Nord  Polling area   129.0       98.0        88.0   
2     1977   1. Nord  Polling area   111.0       90.0        92.0   
3     1978   1. Nord  Polling area   127.0       72.0        77.0   
4     1980   1. Nord  Polling area   128.0      115.0       107.0   
...    ...       ...           ...     ...        ...         ...   
2578  2016  Østerbro      District   461.0    1,877.0     2,905.0   
2579  2017  Østerbro      District   475.0    2,006.0     3,000.0   
2580  2018  Østerbro      District   463.0    2,235.0     3,017.0   
2581  2019  Østerbro      District   443.0    2,277.0     3,018.0   
2582  2020  Østerbro      District   437.0    1,708.0     2,423.0   

      Internal migration  Internal migration from other districts  \
0                  158.0                                    663.0   
1                  110.0                                    622.0   
2                  127.0                                    637.0   
3                  111.0                                    652.0   
4                   99.0                                    686.0   
...                  ...                                      ...   
2578             3,788.0                                  5,533.0   
2579             3,855.0                                  5,642.0   
2580             3,709.0                                  5,310.0   
2581             3,813.0                                  5,493.0   
2582             4,000.0                                  5,784.0   

      Internal migration to to other districts  \
0                                        744.0   
1                                        681.0   
2                                        633.0   
3                                        638.0   
4                                        558.0   
...                                        ...   
2578                                   5,835.0   
2579                                   5,918.0   
2580                                   5,963.0   
2581                                   6,120.0   
2582                                   6,180.0   

      Internal migratrion from danish municipalities  \
0                                              562.0   
1                                              496.0   
2                                              446.0   
3                                              465.0   
4                                              587.0   
...                                              ...   
2578                                         5,576.0   
2579                                         5,818.0   
2580                                         4,930.0   
2581                                         5,255.0   
2582                                         5,574.0   

      Internal migratrion to danish municipalities  Live births  \
0                                            668.0         93.0   
1                                            637.0         73.0   
2                                            564.0         72.0   
3                                            499.0         73.0   
4                                            501.0         75.0   
...                                            ...          ...   
2578                                       5,467.0      1,267.0   
2579                                       5,904.0      1,244.0   
2580                                       5,726.0      1,251.0   
2581                                       5,699.0      1,303.0   
2582                                       6,309.0      1,268.0   

      Natural increase  Netmigration  Netmigration Copenhagen  \
0                -41.0         -38.0                    -81.0   
1                -56.0         -10.0                    -59.0   
2                -39.0           2.0                      4.0   
3        

<hr style="border:2px solid black"> </hr>

# Creation of a superset containing mixed informations from all datasets

We know that we have 1 dataset without the district information: the one that describes the total population of Copenhagen by the country of origin. We will treat this dataset separately.

Now, let's see how much information the dataset containing district information have in common:

## Identify common data categories in dataframes containing district information

In [10]:
# Get the names of all columns in all tables.
all_columns = cph_data.get_all_column_names()

# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get a dict containing the names of dataframes that
# contain a given column.
datasets_with_column = {column: [df.attrs["name"] for df in all_dataframes_with_district
                                 if column in df.columns]
                        for column in all_columns}

# Get the column names sorted by the number of datasets in which they appear.
columns_by_popularity = sorted(datasets_with_column,
                               key=lambda column: len(
                                   datasets_with_column[column]),
                               reverse=True)

# Create dictionary sorted by the the popularity of the column
# (from most to least popular).
datasets_with_column_sorted = {column: datasets_with_column[column]
                               for column in columns_by_popularity}

# Print out the most popular columns.
for column_name, df_names in datasets_with_column_sorted.items():
    print()
    printmd(f'Column **_{column_name}_** occurs in {len(df_names)} '
            f'out of {len(all_dataframes_with_district)} dataframes:')
    print(df_names)

Column **_District_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Column **_District type_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Column **_Year_** occurs in 7 out of 7 dataframes:

...

['Income']



Column **_Total square meters occupied dwellings_** occurs in 1 out of 7 dataframes:

['Dwellings']



Column **_Country of origin_** occurs in 0 out of 7 dataframes:

[]


We can use **Year**, **District**, and **District Type** as basis of a superdataset, where we cross informations from various datasets.

## Steamline districts

For simplicity, it might be beneficial to use only one district type. Let's see which ones we have available:

In [11]:
# Get the names of all columns in all tables.
all_columns = cph_data.get_all_column_names()

# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get all district types.
all_district_types = list()
for df in cph_data.get_dataframes_with_district():
    all_district_types += list(df['District type'].unique())

all_district_types = unique(all_district_types)

# Display the district types.
printmd('All district types:')
for district in all_district_types:
    district_string = '***, ***'.join(all_district_types)

printmd('***' + '***, ***'.join(all_district_types) +  '***')

All district types:

***District***, ***Entire Copenhagen***, ***Polling area***

***Entire Copenhagen*** is not a district, but a summation of all districts in Copenhagen. ***District*** and ***Polling area*** are real districts - ***District*** is of lower resolution and ***Polling area*** is of higher resolution.

In [12]:
# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get all Polling Areas.
all_polling_areas = list()
for df in cph_data.get_dataframes_with_district():
    all_polling_areas += list(df.loc[df['District type'] == 'Polling area', 'District'].unique())
all_polling_areas = unique(all_polling_areas)

# Get all Districts.
all_districts = list()
for df in cph_data.get_dataframes_with_district():
    all_districts += list(df.loc[df['District type'] == 'District', 'District'].unique())
all_districts = unique(all_districts)

# Show the Polling Areas and Districts:
display(pd.DataFrame(sorted(all_polling_areas), columns=['Polling areas']))
display(pd.DataFrame(sorted(all_districts), columns=['Districts']))

Polling areas
0             1. Nord
1         1. Nordvest
2              1. Syd
3             1. Vest
4              1. Øst
5         1. Østerbro
6             2. Nord
7     2. Sundbyvester
8              2. Syd
9             2. Vest
10             2. Øst
11        3. Indre By
12            3. Nord
13             3. Syd
14             3. Øst
15            4. Nord
16     4. Sundbyøster
17             4. Syd
18             4. Øst
19            5. Nord
20        5. Nordvest
21        5. Nørrebro
22  5. Nørrebrohallen
23             5. Syd
24            5. Vest
25             5. Øst
26      6. Bispebjerg
27            6. Nord
28             6. Syd
29            6. Vest
30             6. Øst
31        7. Brønshøj
32      7. Katrinedal
33      7. Kirkebjerg
34            7. Nord
35        7. Nordvest
36             7. Syd
37         7. Vanløse
38            7. Vest
39             7. Øst
40            8. Midt
41            8. Nord
42             8. Syd
43          8. Sydøst
44           8. Valby
45            8. Vest
46            9. Midt
47            9. Nord
48             9. Syd
49         9. Sydhavn
50            9. Vest
51       9. Vesterbro
52             9. Øst

Districts
0                 Amager Vest
1                  Amager Øst
2                  Bispebjerg
3              Brønshøj-Husum
4                    Indre By
5                    Nørrebro
6                   Unlocated
7                       Valby
8                     Vanløse
9   Vesterbro/Kongens Enghave
10                   Østerbro

Note that one of the districts is ***Unlocated***.

Let' now take a look at which datasets have which dataset type:

In [13]:
datasets_with_district_type = {district_type: [df.attrs["name"]
                                               for df in all_dataframes_with_district
                                               if district_type in df['District type'].values]
                               for district_type in all_district_types}

# Get the district names sorted by the number of datasets in which they appear.
district_types_by_popularity = sorted(datasets_with_district_type,
                                      key=lambda column: len(
                                          datasets_with_district_type[column]),
                                      reverse=True)

# Create dictionary sorted by the the popularity of the district type.
# (from most to least popular).
datasets_with_district_type_sorted = {column: datasets_with_district_type[column]
                                      for column in district_types_by_popularity}

# Print out the most popular columns.
for column_name, df_names in datasets_with_district_type_sorted.items():
    print()
    printmd(f'Dataset type **_{column_name}_** occurs in {len(df_names)} '
            f'out of {len(all_dataframes_with_district)} dataframes:')
    print(df_names)

Dataset type **_District_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Dataset type **_Polling area_** occurs in 6 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Marital status', 'Population movement']



Dataset type **_Entire Copenhagen_** occurs in 5 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Marital status']


So, if we use ***District*** as an area, we will be able to divide Copenhagen in 10 districts and use all of our datasets containing district informattion.

If we use ***Polling area***, we will be able to divide Copenhagen in 52 districts and use all of our datasets except the one containing information about *Life span*.

It is a bit of a bummer, because I would like to use the high resolution, but I am also very interested in taking a look on how life span correlates with all the other factors. Therefore, at least as a starting point, I will use ***District***.

## Initialize the superset

Let's create an object for our superset, use only districts of type: ***District*** and drop the column: ***District type***, which becomes superficial.

In [14]:
# Initialize the dataset that will contain the cross of all the
# dataset available in this project.
cph_data_superset = CphData()

# Filter the districts to include only District type: District
# and drop the column "District type"
for attribute_name, df in get_obj_attributes(cph_data_superset).items():
    # Delete all the datasets from the object that do not contain
    # the district data
    if 'District' not in df.columns:
        delattr(cph_data_superset, attribute_name)
    
    # Limit the data to District Type: District and
    # remove the column: District type
    else:
        setattr(
            cph_data_superset,
            attribute_name,
            (df
             .loc[df['District type'] == 'District']
             .drop(['District type'], axis=1))
        )

# Display all dataframes:
cph_data_superset.display_dataframes()


**Citizenship**:

Year  Quarter     District    Sex  Age        Citizenship  \
2173    1980        1  Amager Vest    Men  0-9            Denmark   
2174    1981        1  Amager Vest    Men  0-9            Denmark   
2175    1982        1  Amager Vest    Men  0-9            Denmark   
2176    1983        1  Amager Vest    Men  0-9            Denmark   
2177    1984        1  Amager Vest    Men  0-9            Denmark   
...      ...      ...          ...    ...  ...                ...   
157435  2016        4     Østerbro  Women  90+  Western countries   
157436  2017        4     Østerbro  Women  90+  Western countries   
157437  2018        4     Østerbro  Women  90+  Western countries   
157438  2019        4     Østerbro  Women  90+  Western countries   
157439  2020        4     Østerbro  Women  90+  Western countries   

        Number of people  
2173             1,972.0  
2174             1,917.0  
2175             1,888.0  
2176             1,889.0  
2177             1,856.0  
...                  ...  
157435               3.0  
157436               4.0  
157437               5.0  
157438               6.0  
157439               5.0  

[24600 rows x 7 columns]


**Dwellings**:

Year                   District Dwelling ownership  \
1      1991                   Indre By     Owner-occupied   
2      1991                   Østerbro     Owner-occupied   
3      1991                   Nørrebro     Owner-occupied   
4      1991  Vesterbro/Kongens Enghave     Owner-occupied   
5      1991                      Valby     Owner-occupied   
...     ...                        ...                ...   
11846  2021                    Vanløse            Unknown   
11847  2021             Brønshøj-Husum            Unknown   
11848  2021                 Bispebjerg            Unknown   
11849  2021                 Amager Øst            Unknown   
11850  2021                Amager Vest            Unknown   

       Total square meters occupied dwellings  
1                                   566,699.0  
2                                   635,362.0  
3                                   298,278.0  
4                                   273,584.0  
5                                   551,526.0  
...                                       ...  
11846                                     0.0  
11847                                     0.0  
11848                                     0.0  
11849                                     0.0  
11850                                     0.0  

[1860 rows x 4 columns]


**Family type and children**:

...

Year                   District  Average life span
0    2009                   Indre By               79.4
1    2009                   Østerbro               78.4
2    2009                   Nørrebro               72.3
3    2009  Vesterbro/Kongens Enghave               73.5
4    2009                      Valby               75.9
..    ...                        ...                ...
127  2020             Brønshøj-Husum               78.5
128  2020                 Bispebjerg               77.8
129  2020                 Amager Øst               81.0
130  2020                Amager Vest               78.7
131  2020                  Unlocated                0.0

[132 rows x 3 columns]


**Marital status**:

Year  Quarter     District    Sex  Age Marital status  \
2491    1974        1  Amager Vest    Men  0-9       Divorced   
2492    1975        1  Amager Vest    Men  0-9       Divorced   
2493    1976        1  Amager Vest    Men  0-9       Divorced   
2494    1977        1  Amager Vest    Men  0-9       Divorced   
2495    1978        1  Amager Vest    Men  0-9       Divorced   
...      ...      ...          ...    ...  ...            ...   
240635  2016        4     Østerbro  Women  90+        Widowed   
240636  2017        4     Østerbro  Women  90+        Widowed   
240637  2018        4     Østerbro  Women  90+        Widowed   
240638  2019        4     Østerbro  Women  90+        Widowed   
240639  2020        4     Østerbro  Women  90+        Widowed   

        Number of people  
2491                 0.0  
2492                 0.0  
2493                 0.0  
2494                 0.0  
2495                 0.0  
...                  ...  
240635             235.0  
240636             220.0  
240637             211.0  
240638             208.0  
240639             199.0  

[37600 rows x 7 columns]


**Population movement**:

Year     District  Deaths  Emigrated  Immigrated  Internal migration  \
2173  1975  Amager Vest 1,041.0    1,451.0       564.0             3,454.0   
2174  1976  Amager Vest 1,063.0    1,295.0       720.0             2,935.0   
2175  1977  Amager Vest   965.0    1,689.0       815.0             3,775.0   
2176  1978  Amager Vest   984.0    1,357.0       870.0             2,865.0   
2177  1980  Amager Vest 1,001.0    1,297.0       655.0             2,888.0   
...    ...          ...     ...        ...         ...                 ...   
2578  2016     Østerbro   461.0    1,877.0     2,905.0             3,788.0   
2579  2017     Østerbro   475.0    2,006.0     3,000.0             3,855.0   
2580  2018     Østerbro   463.0    2,235.0     3,017.0             3,709.0   
2581  2019     Østerbro   443.0    2,277.0     3,018.0             3,813.0   
2582  2020     Østerbro   437.0    1,708.0     2,423.0             4,000.0   

      Internal migration from other districts  \
2173                                  6,344.0   
2174                                  6,162.0   
2175                                  6,626.0   
2176                                  5,656.0   
2177                                  5,757.0   
...                                       ...   
2578                                  5,533.0   
2579                                  5,642.0   
2580                                  5,310.0   
2581                                  5,493.0   
2582                                  5,784.0   

      Internal migration to to other districts  \
2173                                   4,817.0   
2174                                   4,942.0   
2175                                   4,828.0   
2176                                   4,561.0   
2177                                   4,355.0   
...                                        ...   
2578                                   5,835.0   
2579                                   5,918.0   
2580                                   5,963.0   
2581                                   6,120.0   
2582                                   6,180.0   

      Internal migratrion from danish municipalities  \
2173                                         2,736.0   
2174                                         2,979.0   
2175                                         2,676.0   
2176                                         2,695.0   
2177                                         2,869.0   
...                                              ...   
2578                                         5,576.0   
2579                                         5,818.0   
2580                                         4,930.0   
2581                                         5,255.0   
2582                                         5,574.0   

      Internal migratrion to danish municipalities  Live births  \
2173                                       3,974.0        559.0   
2174                                       4,211.0        433.0   
2175                                       3,926.0        444.0   
2176                                       3,495.0        460.0   
2177                                       3,007.0        414.0   
...                                            ...          ...   
2578                                       5,467.0      1,267.0   
2579                                       5,904.0      1,244.0   
2580                                       5,726.0      1,251.0   
2581                                       5,699.0      1,303.0   
2582                                       6,309.0      1,268.0   

      Natural increase  Netmigration  Netmigration Copenhagen  \
2173            -482.0        -887.0                  1,527.0   
2174            -630.0        -575.0                  1,220.0   
2175            -521.0        -874.0                  1,798.0   
2176            -524.0        -487.0                  1,095.0   
2177            -587.0        -642.0                  1,402.0   
...        

## Remove the column `Quarter`

As we have already changed the temporal resolution of all the datasets to ***one year***, the column `Quarter` has become superficial.

In [15]:
for attribute_name, df in get_obj_attributes(cph_data_superset).items():
    # Delete all the datasets from the object that do not contain
    # the district data.
    if 'Quarter' in df.columns:
        df.drop(['Quarter'], axis=1, inplace=True)
    
# Display one dataframe to veryfy that the column Quarter is gone:
display(cph_data_superset.family_type_and_children.head(3))

Year     District                                      Family type  \
1219  1998  Amager Vest  Children below 18 years not living with parents   
1220  1999  Amager Vest  Children below 18 years not living with parents   
1221  2000  Amager Vest  Children below 18 years not living with parents   

     Number of children  Number of families  
1219         0 children                82.0  
1220         0 children                78.0  
1221         0 children                77.0

## Prepare each dataset for inclusion in superset

### Citizenship

#### Display

In [16]:
display(cph_data_superset.citizenship)

Year     District    Sex  Age        Citizenship  Number of people
2173    1980  Amager Vest    Men  0-9            Denmark           1,972.0
2174    1981  Amager Vest    Men  0-9            Denmark           1,917.0
2175    1982  Amager Vest    Men  0-9            Denmark           1,888.0
2176    1983  Amager Vest    Men  0-9            Denmark           1,889.0
2177    1984  Amager Vest    Men  0-9            Denmark           1,856.0
...      ...          ...    ...  ...                ...               ...
157435  2016     Østerbro  Women  90+  Western countries               3.0
157436  2017     Østerbro  Women  90+  Western countries               4.0
157437  2018     Østerbro  Women  90+  Western countries               5.0
157438  2019     Østerbro  Women  90+  Western countries               6.0
157439  2020     Østerbro  Women  90+  Western countries               5.0

[24600 rows x 6 columns]

#### Aggregate over columns `Age` and `Sex`

In [17]:
# Aggregate number of people over columns Sex and Age
columns_to_aggregate = ['Number of people']
columns_to_remove = ['Sex', 'Age'] 

# Create a dataframe where Age and Sex information will be removed.
df_temp_citizenship_1 = (
    cph_data_superset.citizenship
    .groupby(get_df_columns(cph_data_superset.citizenship,
                            exclude=columns_to_remove + columns_to_aggregate),
             as_index=False)
    .sum()
)

# Show the result.
display(df_temp_citizenship_1)

Year                   District            Citizenship  Number of people
0     1980                Amager Vest                Denmark          45,262.0
1     1980                Amager Vest  Non-western countries           1,064.0
2     1980                Amager Vest      Western countries           1,502.0
3     1980                 Amager Øst                Denmark          46,257.0
4     1980                 Amager Øst  Non-western countries           1,197.0
...    ...                        ...                    ...               ...
1225  2020  Vesterbro/Kongens Enghave  Non-western countries           4,482.0
1226  2020  Vesterbro/Kongens Enghave      Western countries           8,949.0
1227  2020                   Østerbro                Denmark          68,643.0
1228  2020                   Østerbro  Non-western countries           3,866.0
1229  2020                   Østerbro      Western countries           7,860.0

[1230 rows x 4 columns]

#### Create new columns containing the number of people for each citizenship group:

In [18]:
# Create new columns containing the number of people for each citizenship group:
df_temp_citizenship_2 = (
    df_create_column_for_each_unique_value(
        df_temp_citizenship_1,
        category_columns=['Citizenship'],
        value_columns='Number of people'
    )
)

# Show the results.
display(df_temp_citizenship_2)

Year                   District Number of people                        \
                                              Denmark Non-western countries   
0    1980                Amager Vest         45,262.0               1,064.0   
1    1980                 Amager Øst         46,257.0               1,197.0   
2    1980                 Bispebjerg         43,259.0               1,039.0   
3    1980             Brønshøj-Husum         38,858.0                 299.0   
4    1980                   Indre By         42,261.0                 915.0   
..    ...                        ...              ...                   ...   
405  2020                   Nørrebro         67,274.0               4,953.0   
406  2020                      Valby         51,397.0               5,105.0   
407  2020                    Vanløse         36,293.0               2,248.0   
408  2020  Vesterbro/Kongens Enghave         60,744.0               4,482.0   
409  2020                   Østerbro         68,643.0               3,866.0   

                       
    Western countries  
0             1,502.0  
1             1,019.0  
2               589.0  
3               526.0  
4             2,582.0  
..                ...  
405           7,374.0  
406           5,393.0  
407           2,617.0  
408           8,949.0  
409           7,860.0  

[410 rows x 5 columns]

#### Let's express the number of people as fractions of the total.

In [19]:
# Initialize the processing stage.
df_temp_citizenship_3 = df_temp_citizenship_2.copy()

# Calculate the total number of people
df_temp_citizenship_3['Total number of people'] = (
    df_temp_citizenship_3
    .loc[:, ['Number of people']]
    .sum(axis=1)
)

# Calculate percentages
df_temp_citizenship_3['Number of people'] = (
    df_temp_citizenship_3['Number of people']
    .div(df_temp_citizenship_3['Total number of people'], axis='index')
    .mul(100, axis='index')
)

# Change the name from "number of people" to ""% of people"
df_temp_citizenship_3.columns = (
    df_temp_citizenship_3.columns.set_levels(
        (
            df_temp_citizenship_3
            .columns
            .levels[0]
            .str
            .replace('Number of people', r'% of people')
            .str
            .replace('Total number of people', r'Number of people')
        ),
        level=0
    )
)

# Put assign the dataframe to the object containing
# the data for the superset.
cph_data_superset.citizenship = df_temp_citizenship_3

# Show the resulting dataframe.
display(cph_data_superset.citizenship)

Year                   District % of people                        \
                                         Denmark Non-western countries   
0    1980                Amager Vest        94.6                   2.2   
1    1980                 Amager Øst        95.4                   2.5   
2    1980                 Bispebjerg        96.4                   2.3   
3    1980             Brønshøj-Husum        97.9                   0.8   
4    1980                   Indre By        92.4                   2.0   
..    ...                        ...         ...                   ...   
405  2020                   Nørrebro        84.5                   6.2   
406  2020                      Valby        83.0                   8.2   
407  2020                    Vanløse        88.2                   5.5   
408  2020  Vesterbro/Kongens Enghave        81.9                   6.0   
409  2020                   Østerbro        85.4                   4.8   

                      Number of people  
    Western countries                   
0                 3.1         47,828.0  
1                 2.1         48,473.0  
2                 1.3         44,887.0  
3                 1.3         39,683.0  
4                 5.6         45,758.0  
..                ...              ...  
405               9.3         79,601.0  
406               8.7         61,895.0  
407               6.4         41,158.0  
408              12.1         74,175.0  
409               9.8         80,369.0  

[410 rows x 6 columns]

#### Check that the sum of citizens looks right

In [20]:
# Sum all the numbers of people in 2019.
sum_2019 =(
    cph_data_superset.citizenship
    .loc[cph_data_superset.citizenship['Year'] == 2019,
         'Number of people']
    .to_numpy()
    .sum()
)

# Display the result.
printmd(f'The sum of people in Copenhagen in 2019 '
        f'has been calculated to ***{sum_2019:,.0f}.***')

The sum of people in Copenhagen in 2019 has been calculated to ***629,513.***

### Dwellings

#### Display

In [21]:
display(cph_data_superset.dwellings)

Year                   District Dwelling ownership  \
1      1991                   Indre By     Owner-occupied   
2      1991                   Østerbro     Owner-occupied   
3      1991                   Nørrebro     Owner-occupied   
4      1991  Vesterbro/Kongens Enghave     Owner-occupied   
5      1991                      Valby     Owner-occupied   
...     ...                        ...                ...   
11846  2021                    Vanløse            Unknown   
11847  2021             Brønshøj-Husum            Unknown   
11848  2021                 Bispebjerg            Unknown   
11849  2021                 Amager Øst            Unknown   
11850  2021                Amager Vest            Unknown   

       Total square meters occupied dwellings  
1                                   566,699.0  
2                                   635,362.0  
3                                   298,278.0  
4                                   273,584.0  
5                                   551,526.0  
...                                       ...  
11846                                     0.0  
11847                                     0.0  
11848                                     0.0  
11849                                     0.0  
11850                                     0.0  

[1860 rows x 4 columns]

#### Create new columns containing the number of total of square meters for each dwelling ownership group.

In [22]:
# Create new columns containing the number of people for dwelling ownership' group.
df_temp_dwellings_1 = (
    df_create_column_for_each_unique_value(
        cph_data_superset.dwellings,
        category_columns='Dwelling ownership',
        value_columns='Total square meters occupied dwellings'
    )
)

# Show the results.
display(df_temp_dwellings_1)

Year                   District Total square meters occupied dwellings  \
                                                        Housing cooperative   
0    1991                Amager Vest                              304,751.0   
1    1991                 Amager Øst                              457,312.0   
2    1991                 Bispebjerg                              225,184.0   
3    1991             Brønshøj-Husum                               66,738.0   
4    1991                   Indre By                              334,860.0   
..    ...                        ...                                    ...   
305  2021                   Nørrebro                            1,356,364.0   
306  2021                      Valby                              589,246.0   
307  2021                    Vanløse                              412,189.0   
308  2021  Vesterbro/Kongens Enghave                            1,096,714.0   
309  2021                   Østerbro                            1,322,323.0   

                                                                         \
    Non-profit housing Owner-occupied Private rental Public authorities   
0            363,436.0      422,697.0      656,597.0           47,081.0   
1            279,473.0      525,356.0      588,056.0           97,804.0   
2            600,972.0      243,548.0      397,895.0          315,623.0   
3            540,095.0      572,879.0      234,872.0           92,475.0   
4            123,646.0      566,699.0    1,211,717.0          141,718.0   
..                 ...            ...            ...                ...   
305          687,400.0      305,394.0      707,955.0           55,142.0   
306          493,886.0      790,576.0      642,010.0           13,061.0   
307          202,948.0      669,109.0      391,645.0            2,875.0   
308          459,806.0      667,276.0      912,130.0           21,060.0   
309          368,054.0      989,282.0    1,080,313.0           18,105.0   

             
    Unknown  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4       0.0  
..      ...  
305     0.0  
306     0.0  
307     0.0  
308     0.0  
309     0.0  

[310 rows x 8 columns]

#### Check if there are any square meters in the column: `Unknown`.

In [23]:
# Get the sum of m2 in the column "Unknown".
m2_in_uknown = (
    df_temp_dwellings_1
    .loc[:, ('Total square meters occupied dwellings', 'Unknown')].sum()
)

# Show the result.
printmd(f'The sum of square meters in the column `Unknown` is: ***{m2_in_uknown}***.')

The sum of square meters in the column `Unknown` is: ***0.0***.

Therefore, we will remove the column `Unknown`:

In [24]:
# Remove the column "Unknown":
df_temp_dwellings_2 = (
    df_temp_dwellings_1.drop(
        ('Total square meters occupied dwellings', 'Unknown'),
        axis=1
    )
)

# Show the results.
display(df_temp_dwellings_2)

Year                   District Total square meters occupied dwellings  \
                                                        Housing cooperative   
0    1991                Amager Vest                              304,751.0   
1    1991                 Amager Øst                              457,312.0   
2    1991                 Bispebjerg                              225,184.0   
3    1991             Brønshøj-Husum                               66,738.0   
4    1991                   Indre By                              334,860.0   
..    ...                        ...                                    ...   
305  2021                   Nørrebro                            1,356,364.0   
306  2021                      Valby                              589,246.0   
307  2021                    Vanløse                              412,189.0   
308  2021  Vesterbro/Kongens Enghave                            1,096,714.0   
309  2021                   Østerbro                            1,322,323.0   

                                                                         
    Non-profit housing Owner-occupied Private rental Public authorities  
0            363,436.0      422,697.0      656,597.0           47,081.0  
1            279,473.0      525,356.0      588,056.0           97,804.0  
2            600,972.0      243,548.0      397,895.0          315,623.0  
3            540,095.0      572,879.0      234,872.0           92,475.0  
4            123,646.0      566,699.0    1,211,717.0          141,718.0  
..                 ...            ...            ...                ...  
305          687,400.0      305,394.0      707,955.0           55,142.0  
306          493,886.0      790,576.0      642,010.0           13,061.0  
307          202,948.0      669,109.0      391,645.0            2,875.0  
308          459,806.0      667,276.0      912,130.0           21,060.0  
309          368,054.0      989,282.0    1,080,313.0           18,105.0  

[310 rows x 7 columns]

#### Let's express the number of people as fractions of the total.

In [25]:
# Initialize the processing stage.
df_temp_dwellings_3 = df_temp_dwellings_2.copy()

# Calculate the total number of m2
df_temp_dwellings_3['Total m2'] = (
    df_temp_dwellings_3
    .loc[:, ['Total square meters occupied dwellings']]
    .sum(axis=1)
)

# Calculate percentages.
df_temp_dwellings_3['Total square meters occupied dwellings'] = (
    df_temp_dwellings_3['Total square meters occupied dwellings']
    .div(df_temp_dwellings_3['Total m2'], axis='index')
    .mul(100, axis='index')
)

# Change the name from "Total square meters occupied dwellings"
# to "% of total dwellings area."
df_temp_dwellings_3.columns = (
    df_temp_dwellings_3.columns.set_levels(
        (
            df_temp_dwellings_3
            .columns
            .levels[0]
            .str
            .replace('Total square meters occupied dwellings', r'% of total dwellings area')
            .str
            .replace('Total m2', r'Total dwellings area')
        ),
        level=0
    )
)

# Put assign the dataframe to the object containing
# the data for the superset.
cph_data_superset.dwellings = df_temp_dwellings_3

# Show the resulting dataframe.
display(cph_data_superset.dwellings)

Year                   District % of total dwellings area  \
                                           Housing cooperative   
0    1991                Amager Vest                      17.0   
1    1991                 Amager Øst                      23.5   
2    1991                 Bispebjerg                      12.6   
3    1991             Brønshøj-Husum                       4.4   
4    1991                   Indre By                      14.1   
..    ...                        ...                       ...   
305  2021                   Nørrebro                      43.6   
306  2021                      Valby                      23.3   
307  2021                    Vanløse                      24.6   
308  2021  Vesterbro/Kongens Enghave                      34.7   
309  2021                   Østerbro                      35.0   

                                                                         \
    Non-profit housing Owner-occupied Private rental Public authorities   
0                 20.3           23.6           36.6                2.6   
1                 14.3           27.0           30.2                5.0   
2                 33.7           13.7           22.3               17.7   
3                 35.8           38.0           15.6                6.1   
4                  5.2           23.8           50.9                6.0   
..                 ...            ...            ...                ...   
305               22.1            9.8           22.7                1.8   
306               19.5           31.3           25.4                0.5   
307               12.1           39.9           23.3                0.2   
308               14.6           21.1           28.9                0.7   
309                9.7           26.2           28.6                0.5   

    Total dwellings area  
                          
0            1,794,562.0  
1            1,948,001.0  
2            1,783,222.0  
3            1,507,059.0  
4            2,378,640.0  
..                   ...  
305          3,112,255.0  
306          2,528,779.0  
307          1,678,766.0  
308          3,156,986.0  
309          3,778,077.0  

[310 rows x 8 columns]

### Family type and children

#### Display

In [26]:
display(cph_data_superset.family_type_and_children)

Year     District                                      Family type  \
1219   1998  Amager Vest  Children below 18 years not living with parents   
1220   1999  Amager Vest  Children below 18 years not living with parents   
1221   2000  Amager Vest  Children below 18 years not living with parents   
1222   2001  Amager Vest  Children below 18 years not living with parents   
1223   2002  Amager Vest  Children below 18 years not living with parents   
...     ...          ...                                              ...   
36795  2016     Østerbro                                     Single women   
36796  2017     Østerbro                                     Single women   
36797  2018     Østerbro                                     Single women   
36798  2019     Østerbro                                     Single women   
36799  2020     Østerbro                                     Single women   

         Number of children  Number of families  
1219             0 children                82.0  
1220             0 children                78.0  
1221             0 children                77.0  
1222             0 children                72.0  
1223             0 children                81.0  
...                     ...                 ...  
36795  More than 3 children                21.0  
36796  More than 3 children                20.0  
36797  More than 3 children                21.0  
36798  More than 3 children                19.0  
36799  More than 3 children                18.0  

[5750 rows x 5 columns]

#### Create new columns containing the number of families for each family type and number of children

In [27]:
# Create new columns containing the number of families 
# for each family type and number of children.
df_temp_family_type_and_children_1 = (
    df_create_column_for_each_unique_value(
        cph_data_superset.family_type_and_children,
        category_columns=['Family type', 'Number of children'],
        value_columns='Number of families'
    )
)

# Show the result
display(df_temp_family_type_and_children_1)

Year                   District  \
                                       
                                       
0    1998                Amager Vest   
1    1998                 Amager Øst   
2    1998                 Bispebjerg   
3    1998             Brønshøj-Husum   
4    1998                   Indre By   
..    ...                        ...   
225  2020                   Nørrebro   
226  2020                      Valby   
227  2020                    Vanløse   
228  2020  Vesterbro/Kongens Enghave   
229  2020                   Østerbro   

                                 Number of families                     \
    Children below 18 years not living with parents                      
                                         0 children 1 child 2 children   
0                                              82.0     0.0        0.0   
1                                              62.0     0.0        0.0   
2                                              82.0     0.0        0.0   
3                                              88.0     0.0        0.0   
4                                              52.0     0.0        0.0   
..                                              ...     ...        ...   
225                                            55.0     0.0        0.0   
226                                            57.0     0.0        0.0   
227                                            36.0     0.0        0.0   
228                                            52.0     0.0        0.0   
229                                            83.0     0.0        0.0   

                                     \
                                      
    3 children More than 3 children   
0          0.0                  0.0   
1          0.0                  0.0   
2          0.0                  0.0   
3          0.0                  0.0   
4          0.0                  0.0   
..         ...                  ...   
225        0.0                  0.0   
226        0.0                  0.0   
227        0.0                  0.0   
228        0.0                  0.0   
229        0.0                  0.0   

                                                                       \
    Couples living in consensual union and cohabiting couples           
                                                   0 children 1 child   
0                                              1,996.0          561.0   
1                                              2,617.0          640.0   
2                                              2,116.0          397.0   
3                                              1,129.0          490.0   
4                                              2,374.0          605.0   
..                                                 ...            ...   
225                                            5,358.0        1,539.0   
226                                            3,292.0        1,039.0   
227                                            1,875.0          587.0   
228                                            4,622.0        1,535.0   
229                                            4,453.0        1,321.0   

                                                \
                                                 
    2 children 3 children More than 3 children   
0        294.0       61.0                 16.0   
1        275.0       42.0                  7.0   
2        191.0       29.0                  3.0   
3        320.0       62.0                 11.0   
4        289.0       48.0                 10.0   
..         ...        ...                  ...   
225      791.0      120.0                 48.0   
226      586.0       86.0                 25.0   
227      381.0       81.0                 15.0   
228      781.0      136.0                 11.0   
229      796.0      147.0                 20.0   

                                                                              \
    Married couples and registered partnership          

#### Let's express the number if families as fractions of the total.

In [28]:
# Initialize the processing stage.
df_temp_family_type_and_children_2 = df_temp_family_type_and_children_1.copy()

# Calculate the total number of people.
df_temp_family_type_and_children_2['Total number of families'] = (
    df_temp_family_type_and_children_2
    .loc[:, ['Number of families']]
    .sum(axis=1)
)

# Calculate percentages.
df_temp_family_type_and_children_2['Number of families'] = (
    df_temp_family_type_and_children_2['Number of families']
    .div(df_temp_family_type_and_children_2['Total number of families'], axis='index')
    .mul(100, axis='index')
)

# Change the name from "Number of families'"
# to ""% of families"
df_temp_family_type_and_children_2.columns = (
    df_temp_family_type_and_children_2.columns.set_levels(
        (
            df_temp_family_type_and_children_2
            .columns
            .levels[0]
            .str
            .replace('Number of families', r'% of families')
            .str
            .replace('Total number of families', r'Number of families')
        ),
        level=0
    )
)

# Put assign the dataframe to the object containing
# the data for the superset.
cph_data_superset.family_type_and_children = df_temp_family_type_and_children_2

# Show the resulting dataframe.
display(cph_data_superset.family_type_and_children)

Year                   District  \
                                       
                                       
0    1998                Amager Vest   
1    1998                 Amager Øst   
2    1998                 Bispebjerg   
3    1998             Brønshøj-Husum   
4    1998                   Indre By   
..    ...                        ...   
225  2020                   Nørrebro   
226  2020                      Valby   
227  2020                    Vanløse   
228  2020  Vesterbro/Kongens Enghave   
229  2020                   Østerbro   

                                      % of families                     \
    Children below 18 years not living with parents                      
                                         0 children 1 child 2 children   
0                                               0.3     0.0        0.0   
1                                               0.2     0.0        0.0   
2                                               0.3     0.0        0.0   
3                                               0.4     0.0        0.0   
4                                               0.2     0.0        0.0   
..                                              ...     ...        ...   
225                                             0.1     0.0        0.0   
226                                             0.2     0.0        0.0   
227                                             0.2     0.0        0.0   
228                                             0.1     0.0        0.0   
229                                             0.2     0.0        0.0   

                                     \
                                      
    3 children More than 3 children   
0          0.0                  0.0   
1          0.0                  0.0   
2          0.0                  0.0   
3          0.0                  0.0   
4          0.0                  0.0   
..         ...                  ...   
225        0.0                  0.0   
226        0.0                  0.0   
227        0.0                  0.0   
228        0.0                  0.0   
229        0.0                  0.0   

                                                                       \
    Couples living in consensual union and cohabiting couples           
                                                   0 children 1 child   
0                                                  7.1            2.0   
1                                                  8.6            2.1   
2                                                  7.2            1.3   
3                                                  5.2            2.3   
4                                                  8.2            2.1   
..                                                 ...            ...   
225                                               10.6            3.0   
226                                                9.0            2.8   
227                                                7.9            2.5   
228                                               10.0            3.3   
229                                                9.1            2.7   

                                                \
                                                 
    2 children 3 children More than 3 children   
0          1.1        0.2                  0.1   
1          0.9        0.1                  0.0   
2          0.6        0.1                  0.0   
3          1.5        0.3                  0.1   
4          1.0        0.2                  0.0   
..         ...        ...                  ...   
225        1.6        0.2                  0.1   
226        1.6        0.2                  0.1   
227        1.6        0.3                  0.1   
228        1.7        0.3                  0.0   
229        1.6        0.3                  0.0   

                                                                              \
    Married couples and registered partnership          

#### Check that the number of people looks right

In [29]:
# Sum all the number of people in 2019.
sum_2019 =(
    cph_data_superset.family_type_and_children
    .loc[cph_data_superset.family_type_and_children['Year'] == 2019,
         'Number of families']
    .to_numpy()
    .sum()
)

# Display the result.
printmd(f'The sum of families of Copenhagen in 2019 '
        f'has been calculated to ***{sum_2019:,.0f}.***')

The sum of families of Copenhagen in 2019 has been calculated to ***384,090.***

### Income

#### Display

In [30]:
display(cph_data_superset.income)

Year                   District    Sex  Total income in district (kr.)  \
106   1987                Amager Vest    Men                 2,368,858,000.0   
107   1987                Amager Vest  Women                 2,056,151,000.0   
108   1987                 Amager Øst    Men                 2,614,378,000.0   
109   1987                 Amager Øst  Women                 2,191,694,000.0   
110   1987                 Bispebjerg    Men                 2,127,197,000.0   
...    ...                        ...    ...                             ...   
4285  2019                    Vanløse  Women                 5,566,365,000.0   
4286  2019  Vesterbro/Kongens Enghave    Men                11,307,215,000.0   
4287  2019  Vesterbro/Kongens Enghave  Women                 9,399,658,000.0   
4288  2019                   Østerbro    Men                13,865,000,000.0   
4289  2019                   Østerbro  Women                11,846,349,000.0   

      Average income (kr.)  Number of people  
106              138,659.0          17,084.0  
107              103,898.0          19,790.0  
108              140,520.0          18,605.0  
109              103,348.0          21,207.0  
110              128,679.0          16,531.0  
...                    ...               ...  
4285             319,410.0          17,427.0  
4286             381,678.0          29,625.0  
4287             319,325.0          29,436.0  
4288             448,024.0          30,947.0  
4289             342,786.0          34,559.0  

[726 rows x 6 columns]

#### Aggregate over column `Sex`

In [31]:
# Drop the column: "Average income" as it is not suited for aggregating.
df_temp_income_1 = (
    cph_data_superset.income
    .drop(['Average income (kr.)'], axis=1)
)

# Aggregate number of people over columns Sex and Age
columns_to_aggregate = ['Total income in district (kr.)', 'Number of people']
columns_to_remove = ['Sex'] 

# Create a dataframe where Age and Sex information will be removed.
df_temp_income_1 = (
    df_temp_income_1
    .groupby(get_df_columns(df_temp_income_1,
                            exclude=columns_to_remove + columns_to_aggregate),
             as_index=False)
    .sum()
)

# Re-add the column: "Average income".
df_temp_income_1['Average income (kr.)'] = (
    df_temp_income_1['Total income in district (kr.)']
    .div(df_temp_income_1['Number of people'])
)

# Sort the columns.
df_temp_income_1 = df_sort_columns(df_temp_income_1)

# Show the result.
display(df_temp_income_1)

Year                   District  Average income (kr.)  Number of people  \
0    1987                Amager Vest             120,003.5          36,874.0   
1    1987                 Amager Øst             120,719.2          39,812.0   
2    1987                 Bispebjerg             109,902.1          37,841.0   
3    1987             Brønshøj-Husum             124,875.5          31,076.0   
4    1987                   Indre By             146,519.9          35,839.0   
..    ...                        ...                   ...               ...   
358  2019                  Unlocated             144,593.6           2,955.0   
359  2019                      Valby             331,929.4          48,587.0   
360  2019                    Vanløse             350,026.9          33,675.0   
361  2019  Vesterbro/Kongens Enghave             350,601.5          59,061.0   
362  2019                   Østerbro             392,503.7          65,506.0   

     Total income in district (kr.)  
0                   4,425,009,000.0  
1                   4,806,072,000.0  
2                   4,158,804,000.0  
3                   3,880,632,000.0  
4                   5,251,126,000.0  
..                              ...  
358                   427,274,000.0  
359                16,127,455,000.0  
360                11,787,157,000.0  
361                20,706,873,000.0  
362                25,711,349,000.0  

[363 rows x 5 columns]

#### Check that the number of people looks right

In [32]:
# Sum all the number of people in 2019.
sum_2019 =(
    df_temp_income_1
    .loc[df_temp_income_1['Year'] == 2019,
         'Number of people']
    .to_numpy()
    .sum()
)

# Display the result.
printmd(f'The sum of people in Copenhagen in 2019 '
        f'has been calculated to ***{sum_2019:,.0f}.***')

The sum of people in Copenhagen in 2019 has been calculated to ***515,890.***

This number is significantly lower than the ***629,513*** we get from the ***Citizenship*** dataset. Let's take a look why:

In [33]:
# Define the relevant columns.
columns = ['Year', 'District', 'Number of people']

# Pre process the dataframes.
df_n_people_citizenship = cph_data_superset.citizenship[columns]
df_n_people_citizenship.columns = df_n_people_citizenship.columns.droplevel(1)

# Create a dataframe showing the number of people by year and district
# for the Citizenship and Income dataset.
df_n_people = (
    df_n_people_citizenship
    .merge(cph_data_superset.income[columns],
           on=['Year', 'District'],
           suffixes=[', ' + cph_data_superset.citizenship.attrs['name'],
                     ', ' + cph_data_superset.income.attrs['name']],
           how='inner')
)

# Add a column showing the difference in the number of people.
df_n_people['Number of people, Citizenship - Income'] = (
    df_n_people['Number of people, Citizenship']
    .sub(df_n_people['Number of people, Income'])
)

# Display the results.
display(df_n_people)
printmd('Statistics about the difference:')
display(df_n_people['Number of people, Citizenship - Income']
        .describe()
        .to_frame())

Year                   District  Number of people, Citizenship  \
0    1987                Amager Vest                       45,978.0   
1    1987                Amager Vest                       45,978.0   
2    1987                 Amager Øst                       45,879.0   
3    1987                 Amager Øst                       45,879.0   
4    1987                 Bispebjerg                       43,004.0   
..    ...                        ...                            ...   
655  2019                    Vanløse                       41,137.0   
656  2019  Vesterbro/Kongens Enghave                       72,128.0   
657  2019  Vesterbro/Kongens Enghave                       72,128.0   
658  2019                   Østerbro                       79,908.0   
659  2019                   Østerbro                       79,908.0   

     Number of people, Income  Number of people, Citizenship - Income  
0                    17,084.0                                28,894.0  
1                    19,790.0                                26,188.0  
2                    18,605.0                                27,274.0  
3                    21,207.0                                24,672.0  
4                    16,531.0                                26,473.0  
..                        ...                                     ...  
655                  17,427.0                                23,710.0  
656                  29,625.0                                42,503.0  
657                  29,436.0                                42,692.0  
658                  30,947.0                                48,961.0  
659                  34,559.0                                45,349.0  

[660 rows x 5 columns]

Statistics about the difference:

Number of people, Citizenship - Income
count                                   660.0
mean                                 30,049.3
std                                   7,338.8
min                                  17,930.0
25%                                  24,801.2
50%                                  27,824.0
75%                                  34,145.0
max                                  48,961.0

We can see that there are around 8,500, or around 20% less citizens in each district in the ***Income*** dataset than in the ***Citizenship*** dataset. I believe the difference caused by exclusion of children from the ***Income*** data.

#### Remove columns containing total income and number of people per district.

Because we already know the total number of people in Copenhagen from the ***Citizenship*** dataset, I choose to only keep the information about *average income*, for every year and district.

In [34]:
# Remove columns containing total income and number of people per district.
cph_data_superset.income = (
    df_temp_income_1
    .drop(['Number of people', 'Total income in district (kr.)'],
          axis=1)
)

# Show the result.
display(cph_data_superset.income)

Year                   District  Average income (kr.)
0    1987                Amager Vest             120,003.5
1    1987                 Amager Øst             120,719.2
2    1987                 Bispebjerg             109,902.1
3    1987             Brønshøj-Husum             124,875.5
4    1987                   Indre By             146,519.9
..    ...                        ...                   ...
358  2019                  Unlocated             144,593.6
359  2019                      Valby             331,929.4
360  2019                    Vanløse             350,026.9
361  2019  Vesterbro/Kongens Enghave             350,601.5
362  2019                   Østerbro             392,503.7

[363 rows x 3 columns]

### Lifespan

#### Display

In [35]:
display(cph_data_superset.life_span)

Year                   District  Average life span
0    2009                   Indre By               79.4
1    2009                   Østerbro               78.4
2    2009                   Nørrebro               72.3
3    2009  Vesterbro/Kongens Enghave               73.5
4    2009                      Valby               75.9
..    ...                        ...                ...
127  2020             Brønshøj-Husum               78.5
128  2020                 Bispebjerg               77.8
129  2020                 Amager Øst               81.0
130  2020                Amager Vest               78.7
131  2020                  Unlocated                0.0

[132 rows x 3 columns]

### Marital status

#### Display

In [36]:
display(cph_data_superset.marital_status)

Year     District    Sex  Age Marital status  Number of people
2491    1974  Amager Vest    Men  0-9       Divorced               0.0
2492    1975  Amager Vest    Men  0-9       Divorced               0.0
2493    1976  Amager Vest    Men  0-9       Divorced               0.0
2494    1977  Amager Vest    Men  0-9       Divorced               0.0
2495    1978  Amager Vest    Men  0-9       Divorced               0.0
...      ...          ...    ...  ...            ...               ...
240635  2016     Østerbro  Women  90+        Widowed             235.0
240636  2017     Østerbro  Women  90+        Widowed             220.0
240637  2018     Østerbro  Women  90+        Widowed             211.0
240638  2019     Østerbro  Women  90+        Widowed             208.0
240639  2020     Østerbro  Women  90+        Widowed             199.0

[37600 rows x 6 columns]

#### Aggregate over columns `Age` and `Sex`

In [37]:
# Aggregate number of people over columns Sex and Age
columns_to_aggregate = ['Number of people']
columns_to_remove = ['Sex', 'Age'] 

# Create a dataframe where Age and Sex information will be removed.
df_temp_marital_status_1 = (
    cph_data_superset.marital_status
    .groupby(get_df_columns(cph_data_superset.marital_status,
                            exclude=columns_to_remove + columns_to_aggregate),
             as_index=False)
    .sum()
)

# Show the result.
display(df_temp_marital_status_1)

Year                   District       Marital status  Number of people
0     1974                Amager Vest             Divorced           5,066.0
1     1974                Amager Vest  Married / seperated          22,022.0
2     1974                Amager Vest        Never married          22,094.0
3     1974                Amager Vest              Widowed           4,838.0
4     1974                 Amager Øst             Divorced           5,064.0
...    ...                        ...                  ...               ...
1875  2020  Vesterbro/Kongens Enghave              Widowed           1,027.0
1876  2020                   Østerbro             Divorced           7,375.0
1877  2020                   Østerbro  Married / seperated          19,476.0
1878  2020                   Østerbro        Never married          51,514.0
1879  2020                   Østerbro              Widowed           2,002.0

[1880 rows x 4 columns]

#### Create new columns containing the number of people for each marital status:

In [38]:
# Create new columns containing the number of people for each citizenship group:
df_temp_marital_status_2 = (
    df_create_column_for_each_unique_value(
        df_temp_marital_status_1,
        category_columns=['Marital status'],
        value_columns='Number of people'
    )
)

# Show the results.
display(df_temp_marital_status_2)

Year                   District Number of people                      \
                                             Divorced Married / seperated   
0    1974                Amager Vest          5,066.0            22,022.0   
1    1974                 Amager Øst          5,064.0            25,077.0   
2    1974                 Bispebjerg          5,357.0            24,056.0   
3    1974             Brønshøj-Husum          3,460.0            21,320.0   
4    1974                   Indre By          6,399.0            19,686.0   
..    ...                        ...              ...                 ...   
465  2020                   Nørrebro          5,859.0            14,183.0   
466  2020                      Valby          5,445.0            15,080.0   
467  2020                    Vanløse          3,741.0            11,432.0   
468  2020  Vesterbro/Kongens Enghave          5,999.0            15,325.0   
469  2020                   Østerbro          7,375.0            19,476.0   

                           
    Never married Widowed  
0        22,094.0 4,838.0  
1        18,434.0 5,452.0  
2        15,658.0 6,239.0  
3        16,313.0 3,661.0  
4        24,515.0 3,946.0  
..            ...     ...  
465      58,223.0 1,336.0  
466      39,733.0 1,637.0  
467      24,833.0 1,152.0  
468      51,824.0 1,027.0  
469      51,514.0 2,002.0  

[470 rows x 6 columns]

#### Let's express the number of people as fractions of the total.

In [39]:
# Initialize the processing stage.
df_temp_marital_status_3 = df_temp_marital_status_2.copy()

# Calculate the total number of people
df_temp_marital_status_3['Total number of people'] = (
    df_temp_marital_status_3
    .loc[:, ['Number of people']]
    .sum(axis=1)
)

# Calculate percentages.
df_temp_marital_status_3['Number of people'] = (
    df_temp_marital_status_3['Number of people']
    .div(df_temp_marital_status_3['Total number of people'], axis='index')
    .mul(100, axis='index')
)

# Change the name from "number of people" to ""% of people"
df_temp_marital_status_3.columns = (
    df_temp_marital_status_3.columns.set_levels(
        (
            df_temp_marital_status_3
            .columns
            .levels[0]
            .str
            .replace('Number of people', r'% of people')
            .str
            .replace('Total number of people', r'Number of people')
        ),
        level=0
    )
)

# Show the resulting dataframe.
display(df_temp_marital_status_3)

Year                   District % of people                      \
                                        Divorced Married / seperated   
0    1974                Amager Vest         9.4                40.8   
1    1974                 Amager Øst         9.4                46.4   
2    1974                 Bispebjerg        10.4                46.9   
3    1974             Brønshøj-Husum         7.7                47.6   
4    1974                   Indre By        11.7                36.1   
..    ...                        ...         ...                 ...   
465  2020                   Nørrebro         7.4                17.8   
466  2020                      Valby         8.8                24.4   
467  2020                    Vanløse         9.1                27.8   
468  2020  Vesterbro/Kongens Enghave         8.1                20.7   
469  2020                   Østerbro         9.2                24.2   

                          Number of people  
    Never married Widowed                   
0            40.9     9.0         54,020.0  
1            34.1    10.1         54,027.0  
2            30.5    12.2         51,310.0  
3            36.5     8.2         44,754.0  
4            44.9     7.2         54,546.0  
..            ...     ...              ...  
465          73.1     1.7         79,601.0  
466          64.2     2.6         61,895.0  
467          60.3     2.8         41,158.0  
468          69.9     1.4         74,175.0  
469          64.1     2.5         80,367.0  

[470 rows x 7 columns]

#### Check that the number of people looks right

In [40]:
# Sum all the number of people in 2019.
sum_2019 =(
    df_temp_marital_status_3
    .loc[df_temp_marital_status_3['Year'] == 2019,
         'Number of people']
    .to_numpy()
    .sum()
)

# Display the result.
printmd(f'The sum of people in Copenhagen in 2019 '
        f'has been calculated to ***{sum_2019:,.0f}.***')

The sum of people in Copenhagen in 2019 has been calculated to ***629,513.***

The number is the same as the one obtained from the ***Citizenship*** dataset. We can therefore drop the column `Number of people` and avoid redundancy.

#### Remove the column `Number of people`

In [41]:
# Remove the column "Number of people".
cph_data_superset.marital_status = (
    df_temp_marital_status_3
    .drop(['Number of people'],
          axis=1)
)

# Show the results.
display(cph_data_superset.marital_status)

C:\Users\wojci\.conda\envs\DataScience\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Year                   District % of people                      \
                                        Divorced Married / seperated   
0    1974                Amager Vest         9.4                40.8   
1    1974                 Amager Øst         9.4                46.4   
2    1974                 Bispebjerg        10.4                46.9   
3    1974             Brønshøj-Husum         7.7                47.6   
4    1974                   Indre By        11.7                36.1   
..    ...                        ...         ...                 ...   
465  2020                   Nørrebro         7.4                17.8   
466  2020                      Valby         8.8                24.4   
467  2020                    Vanløse         9.1                27.8   
468  2020  Vesterbro/Kongens Enghave         8.1                20.7   
469  2020                   Østerbro         9.2                24.2   

                           
    Never married Widowed  
0            40.9     9.0  
1            34.1    10.1  
2            30.5    12.2  
3            36.5     8.2  
4            44.9     7.2  
..            ...     ...  
465          73.1     1.7  
466          64.2     2.6  
467          60.3     2.8  
468          69.9     1.4  
469          64.1     2.5  

[470 rows x 6 columns]

### Population movement

#### Display

In [42]:
display(cph_data_superset.population_movement)

Year     District  Deaths  Emigrated  Immigrated  Internal migration  \
2173  1975  Amager Vest 1,041.0    1,451.0       564.0             3,454.0   
2174  1976  Amager Vest 1,063.0    1,295.0       720.0             2,935.0   
2175  1977  Amager Vest   965.0    1,689.0       815.0             3,775.0   
2176  1978  Amager Vest   984.0    1,357.0       870.0             2,865.0   
2177  1980  Amager Vest 1,001.0    1,297.0       655.0             2,888.0   
...    ...          ...     ...        ...         ...                 ...   
2578  2016     Østerbro   461.0    1,877.0     2,905.0             3,788.0   
2579  2017     Østerbro   475.0    2,006.0     3,000.0             3,855.0   
2580  2018     Østerbro   463.0    2,235.0     3,017.0             3,709.0   
2581  2019     Østerbro   443.0    2,277.0     3,018.0             3,813.0   
2582  2020     Østerbro   437.0    1,708.0     2,423.0             4,000.0   

      Internal migration from other districts  \
2173                                  6,344.0   
2174                                  6,162.0   
2175                                  6,626.0   
2176                                  5,656.0   
2177                                  5,757.0   
...                                       ...   
2578                                  5,533.0   
2579                                  5,642.0   
2580                                  5,310.0   
2581                                  5,493.0   
2582                                  5,784.0   

      Internal migration to to other districts  \
2173                                   4,817.0   
2174                                   4,942.0   
2175                                   4,828.0   
2176                                   4,561.0   
2177                                   4,355.0   
...                                        ...   
2578                                   5,835.0   
2579                                   5,918.0   
2580                                   5,963.0   
2581                                   6,120.0   
2582                                   6,180.0   

      Internal migratrion from danish municipalities  \
2173                                         2,736.0   
2174                                         2,979.0   
2175                                         2,676.0   
2176                                         2,695.0   
2177                                         2,869.0   
...                                              ...   
2578                                         5,576.0   
2579                                         5,818.0   
2580                                         4,930.0   
2581                                         5,255.0   
2582                                         5,574.0   

      Internal migratrion to danish municipalities  Live births  \
2173                                       3,974.0        559.0   
2174                                       4,211.0        433.0   
2175                                       3,926.0        444.0   
2176                                       3,495.0        460.0   
2177                                       3,007.0        414.0   
...                                            ...          ...   
2578                                       5,467.0      1,267.0   
2579                                       5,904.0      1,244.0   
2580                                       5,726.0      1,251.0   
2581                                       5,699.0      1,303.0   
2582                                       6,309.0      1,268.0   

      Natural increase  Netmigration  Netmigration Copenhagen  \
2173            -482.0        -887.0                  1,527.0   
2174            -630.0        -575.0                  1,220.0   
2175            -521.0        -874.0                  1,798.0   
2176            -524.0        -487.0                  1,095.0   
2177            -587.0        -642.0                  1,402.0   
...        

#### Let's express the number of people as fractions of the total.

In [43]:
# Initialize the processing stage.
df_temp_population_movement_1 = cph_data_superset.population_movement.copy()

# Get the Number of people from the Citizenship dataset
columns = ['Year', 'District', 'Number of people']
df_n_people_citizenship = cph_data_superset.citizenship[columns]
df_n_people_citizenship.columns = df_n_people_citizenship.columns.droplevel(1)

df_temp_population_movement_1 = (
    df_temp_population_movement_1
    .merge(df_n_people_citizenship,
           on=['Year', 'District'],
           how='inner')
)

# Set the name again, as merge deletes it.
df_temp_population_movement_1.attrs['name'] = 'Population movement'

# Get the columns to express as fractions
columns_to_express_as_fractions = get_df_columns(
    df_temp_population_movement_1,
    exclude=['Year', 'District', 'Number of people']
)

# Calculate percentages.
df_temp_population_movement_1[columns_to_express_as_fractions] = (
    df_temp_population_movement_1[columns_to_express_as_fractions]
    .div(df_temp_population_movement_1['Number of people'], axis='index')
    .mul(100, axis='index')
)

# Drop the column "Number of people".
df_temp_population_movement_1 = (
    df_temp_population_movement_1
    .drop(['Number of people'],
          axis=1)
)

# Adjust the column names.
df_temp_population_movement_1.columns = pd.MultiIndex.from_tuples(
            [(column, ) + ('', )
             if column not in columns_to_express_as_fractions
             else
             (r'% of people', ) + (column, )
             for column in list(df_temp_population_movement_1.columns)]
)

# Assign the dataframe to the superset object.
cph_data_superset.population_movement = df_temp_population_movement_1
    
# Show the resulting dataframe.
display(cph_data_superset.population_movement)

Year     District % of people                                          \
                            Deaths Emigrated Immigrated Internal migration   
0    1980  Amager Vest         2.1       2.7        1.4                6.0   
1    1981  Amager Vest         2.2       2.2        1.1                6.0   
2    1982  Amager Vest         1.9       2.1        1.0                5.8   
3    1987  Amager Vest         1.8       2.0        1.0                6.5   
4    1988  Amager Vest         1.9       2.2        1.0                6.6   
..    ...          ...         ...       ...        ...                ...   
365  2016     Østerbro         0.6       2.4        3.7                4.9   
366  2017     Østerbro         0.6       2.5        3.8                4.9   
367  2018     Østerbro         0.6       2.8        3.8                4.7   
368  2019     Østerbro         0.6       2.8        3.8                4.8   
369  2020     Østerbro         0.5       2.1        3.0                5.0   

                                             \
    Internal migration from other districts   
0                                      12.0   
1                                      11.5   
2                                      12.0   
3                                      13.2   
4                                      14.3   
..                                      ...   
365                                     7.1   
366                                     7.1   
367                                     6.7   
368                                     6.9   
369                                     7.2   

                                              \
    Internal migration to to other districts   
0                                        9.1   
1                                        8.8   
2                                        8.9   
3                                       10.6   
4                                       11.4   
..                                       ...   
365                                      7.5   
366                                      7.5   
367                                      7.5   
368                                      7.7   
369                                      7.7   

                                                    \
    Internal migratrion from danish municipalities   
0                                              6.0   
1                                              5.8   
2                                              5.9   
3                                              6.4   
4                                              6.3   
..                                             ...   
365                                            7.2   
366                                            7.3   
367                                            6.2   
368                                            6.6   
369                                            6.9   

                                                                               \
    Internal migratrion to danish municipalities Live births Natural increase   
0                                            6.3         0.9             -1.2   
1                                            5.9         0.7             -1.5   
2                                            6.0         0.9             -1.1   
3                                            8.1         1.0             -0.9   
4                                            7.0         1.0             -0.9   
..                                           ...         ...              ...   
365                                          7.0         1.6              1.0   
366                                          7.4         1.6              1.0   
367                                          7.2         1.6              1.0   
368                                          7.1         1.6              1.1   
369                                          7.9         1.6              1.0 

## Make sure all the tables have the same number of levels in the column index

In [44]:
# Get the maximal number of levels present in the column index of
# any of the dataframes in the superset.
max_n_levels = max([df.columns.nlevels
                    for df in cph_data_superset.get_all_dataframes()])

# Add empty index layers to columns with less than max levels.
for attribute_name, df in get_obj_attributes(cph_data_superset).items():
    if df.columns.nlevels < max_n_levels:
        df.columns = pd.MultiIndex.from_tuples(
            [idx + ('', ) * (max_n_levels - df.columns.nlevels)
             if isinstance(idx, tuple)
             else (idx, ) + ('', ) * (max_n_levels - df.columns.nlevels)
             for idx in list(df.columns)]
        )
        
    # Save the dataframe in the object: cph_data_superset.
    setattr(cph_data_superset, attribute_name, df)

## Create the superset

### Show all datasets to be merged

In [45]:
cph_data_superset.display_dataframes()


**Citizenship**:

Year                   District % of people                        \
                                         Denmark Non-western countries   
                                                                         
0    1980                Amager Vest        94.6                   2.2   
1    1980                 Amager Øst        95.4                   2.5   
2    1980                 Bispebjerg        96.4                   2.3   
3    1980             Brønshøj-Husum        97.9                   0.8   
4    1980                   Indre By        92.4                   2.0   
..    ...                        ...         ...                   ...   
405  2020                   Nørrebro        84.5                   6.2   
406  2020                      Valby        83.0                   8.2   
407  2020                    Vanløse        88.2                   5.5   
408  2020  Vesterbro/Kongens Enghave        81.9                   6.0   
409  2020                   Østerbro        85.4                   4.8   

                      Number of people  
    Western countries                   
                                        
0                 3.1         47,828.0  
1                 2.1         48,473.0  
2                 1.3         44,887.0  
3                 1.3         39,683.0  
4                 5.6         45,758.0  
..                ...              ...  
405               9.3         79,601.0  
406               8.7         61,895.0  
407               6.4         41,158.0  
408              12.1         74,175.0  
409               9.8         80,369.0  

[410 rows x 6 columns]


**Dwellings**:

Year                   District % of total dwellings area  \
                                           Housing cooperative   
                                                                 
0    1991                Amager Vest                      17.0   
1    1991                 Amager Øst                      23.5   
2    1991                 Bispebjerg                      12.6   
3    1991             Brønshøj-Husum                       4.4   
4    1991                   Indre By                      14.1   
..    ...                        ...                       ...   
305  2021                   Nørrebro                      43.6   
306  2021                      Valby                      23.3   
307  2021                    Vanløse                      24.6   
308  2021  Vesterbro/Kongens Enghave                      34.7   
309  2021                   Østerbro                      35.0   

                                                                         \
    Non-profit housing Owner-occupied Private rental Public authorities   
                                                                          
0                 20.3           23.6           36.6                2.6   
1                 14.3           27.0           30.2                5.0   
2                 33.7           13.7           22.3               17.7   
3                 35.8           38.0           15.6                6.1   
4                  5.2           23.8           50.9                6.0   
..                 ...            ...            ...                ...   
305               22.1            9.8           22.7                1.8   
306               19.5           31.3           25.4                0.5   
307               12.1           39.9           23.3                0.2   
308               14.6           21.1           28.9                0.7   
309                9.7           26.2           28.6                0.5   

    Total dwellings area  
                          
                          
0            1,794,562.0  
1            1,948,001.0  
2            1,783,222.0  
3            1,507,059.0  
4            2,378,640.0  
..                   ...  
305          3,112,255.0  
306          2,528,779.0  
307          1,678,766.0  
308          3,156,986.0  
309          3,778,077.0  

[310 rows x 8 columns]


**Family type and children**:

...

Year                   District Average life span
                                                      
                                                      
0    2009                   Indre By              79.4
1    2009                   Østerbro              78.4
2    2009                   Nørrebro              72.3
3    2009  Vesterbro/Kongens Enghave              73.5
4    2009                      Valby              75.9
..    ...                        ...               ...
127  2020             Brønshøj-Husum              78.5
128  2020                 Bispebjerg              77.8
129  2020                 Amager Øst              81.0
130  2020                Amager Vest              78.7
131  2020                  Unlocated               0.0

[132 rows x 3 columns]


**Marital status**:

Year                   District % of people                      \
                                        Divorced Married / seperated   
                                                                       
0    1974                Amager Vest         9.4                40.8   
1    1974                 Amager Øst         9.4                46.4   
2    1974                 Bispebjerg        10.4                46.9   
3    1974             Brønshøj-Husum         7.7                47.6   
4    1974                   Indre By        11.7                36.1   
..    ...                        ...         ...                 ...   
465  2020                   Nørrebro         7.4                17.8   
466  2020                      Valby         8.8                24.4   
467  2020                    Vanløse         9.1                27.8   
468  2020  Vesterbro/Kongens Enghave         8.1                20.7   
469  2020                   Østerbro         9.2                24.2   

                           
    Never married Widowed  
                           
0            40.9     9.0  
1            34.1    10.1  
2            30.5    12.2  
3            36.5     8.2  
4            44.9     7.2  
..            ...     ...  
465          73.1     1.7  
466          64.2     2.6  
467          60.3     2.8  
468          69.9     1.4  
469          64.1     2.5  

[470 rows x 6 columns]


**Population movement**:

Year     District % of people                                          \
                            Deaths Emigrated Immigrated Internal migration   
                                                                             
0    1980  Amager Vest         2.1       2.7        1.4                6.0   
1    1981  Amager Vest         2.2       2.2        1.1                6.0   
2    1982  Amager Vest         1.9       2.1        1.0                5.8   
3    1987  Amager Vest         1.8       2.0        1.0                6.5   
4    1988  Amager Vest         1.9       2.2        1.0                6.6   
..    ...          ...         ...       ...        ...                ...   
365  2016     Østerbro         0.6       2.4        3.7                4.9   
366  2017     Østerbro         0.6       2.5        3.8                4.9   
367  2018     Østerbro         0.6       2.8        3.8                4.7   
368  2019     Østerbro         0.6       2.8        3.8                4.8   
369  2020     Østerbro         0.5       2.1        3.0                5.0   

                                             \
    Internal migration from other districts   
                                              
0                                      12.0   
1                                      11.5   
2                                      12.0   
3                                      13.2   
4                                      14.3   
..                                      ...   
365                                     7.1   
366                                     7.1   
367                                     6.7   
368                                     6.9   
369                                     7.2   

                                              \
    Internal migration to to other districts   
                                               
0                                        9.1   
1                                        8.8   
2                                        8.9   
3                                       10.6   
4                                       11.4   
..                                       ...   
365                                      7.5   
366                                      7.5   
367                                      7.5   
368                                      7.7   
369                                      7.7   

                                                    \
    Internal migratrion from danish municipalities   
                                                     
0                                              6.0   
1                                              5.8   
2                                              5.9   
3                                              6.4   
4                                              6.3   
..                                             ...   
365                                            7.2   
366                                            7.3   
367                                            6.2   
368                                            6.6   
369                                            6.9   

                                                                               \
    Internal migratrion to danish municipalities Live births Natural increase   
                                                                                
0                                            6.3         0.9             -1.2   
1                                            5.9         0.7             -1.5   
2                                            6.0         0.9             -1.1   
3                                            8.1         1.0             -0.9   
4                                            7.0         1.0             -0.9   
..                                           ...         ...              ...   
365                                          7.0         1.6              1.0   
366          

###  Merge

In [46]:
# Merge 
for idx_df, df in enumerate(cph_data_superset.get_all_dataframes()):
    # The first dataframe is left unaltered.
    if idx_df == 0:
        df_superset = df
        
    # Each consecutive dataframe is merged with the merge of the previous
    # dataframes on columns "Year" and "District".
    else:
        df_superset = df_superset.merge(
            df,
            on=['Year', 'District'],
            how='inner',
        )

# Show the results.
printmd('The **superset**:')
display(df_superset)
display(pd.DataFrame(df_superset['Year'].unique(), columns=['Included years']))
display(pd.DataFrame(df_superset['District'].unique(), columns=['Included districts']))

C:\Users\wojci\.conda\envs\DataScience\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


The **superset**:

Year                   District % of people                        \
                                         Denmark Non-western countries   
                                                                         
0    2009                Amager Vest        86.9                   6.4   
1    2009                 Amager Øst        87.9                   6.0   
2    2009                 Bispebjerg        84.0                  10.4   
3    2009             Brønshøj-Husum        84.7                  11.2   
4    2009                   Indre By        86.9                   2.9   
..    ...                        ...         ...                   ...   
105  2019                   Nørrebro        84.2                   6.5   
106  2019                      Valby        83.1                   8.3   
107  2019                    Vanløse        88.0                   5.6   
108  2019  Vesterbro/Kongens Enghave        81.9                   5.9   
109  2019                   Østerbro        85.2                   4.9   

                      Number of people % of total dwellings area  \
    Western countries                        Housing cooperative   
                                                                   
0                 6.7         56,257.0                      27.2   
1                 6.1         50,390.0                      35.2   
2                 5.6         47,798.0                      32.2   
3                 4.1         39,771.0                      10.1   
4                10.2         47,339.0                      22.7   
..                ...              ...                       ...   
105               9.2         80,646.0                      44.5   
106               8.6         59,840.0                      25.5   
107               6.4         41,137.0                      24.9   
108              12.2         72,128.0                      37.3   
109              10.0         79,908.0                      36.0   

                                                                         \
    Non-profit housing Owner-occupied Private rental Public authorities   
                                                                          
0                 20.1           29.5           22.3                0.8   
1                 14.7           29.3           20.3                0.5   
2                 34.1           15.5           17.9                0.3   
3                 39.3           39.5            9.7                1.4   
4                  5.4           28.1           39.1                4.6   
..                 ...            ...            ...                ...   
105               22.4            9.6           21.8                1.7   
106               19.9           30.3           23.8                0.5   
107               12.0           39.6           23.4                0.2   
108               15.4           20.3           26.3                0.7   
109                9.2           26.1           28.2                0.5   

    Total dwellings area                                   % of families  \
                         Children below 18 years not living with parents   
                                                              0 children   
0            2,469,807.0                                             0.2   
1            2,107,253.0                                             0.2   
2            1,951,637.0                                             0.2   
3            1,628,317.0                                             0.3   
4            2,710,400.0                                             0.2   
..                   ...                                             ...   
105          3,064,700.0                                             0.1   
106          2,341,961.0                                             0.1   
107          1,668,619.0                                             0.2   
108          2,935,055.0                            

Included years
0            2009
1            2010
2            2011
3            2012
4            2013
5            2014
6            2015
7            2016
8            2017
9            2018
10           2019

Included districts
0                Amager Vest
1                 Amager Øst
2                 Bispebjerg
3             Brønshøj-Husum
4                   Indre By
5                   Nørrebro
6                      Valby
7                    Vanløse
8  Vesterbro/Kongens Enghave
9                   Østerbro

### Sort columns

In [47]:
# Sort the columns so that Year and District show up first
df_superset = (
    df_sort_columns(df_superset,
                    first_columns=[('Year', '', ''),
                                   ('District', '', '')])
)

### Save superset dataframe

In [49]:
# Save the superset to disk.
df_superset.to_pickle(path_data_clean_root / 'cph_clean_superset.pkl')

### Show data for a chosen district

In [ ]:
# Choose district
district = 'Nørrebro'

# Sort the columns so that Year and District show up first
display(
    df_superset
    .loc[df_superset[('District', '', '')] == district]
)

<hr style="border:2px solid black"> </hr>

# Sandbox